En esta parte continuaremos con la construcción de un **modelo del lenguaje**.

### Construcción de un modelo del lenguaje

#### ¿Qué es un modelo del lenguaje?

Un modelo del lenguaje es un mecanismo para definir la estructura del lenguaje, es decir, para restringir adecuadamente las secuencias de unidades lingüisticas más probables. En nuestro caso construireos un modelo del lenguaje estadístico, el cual asignará una probabilidad a una secuencia *m* de palabras. Para la construccion de nuestro modelo conformaremos un trigrama de palabras.

#### ¿Qué es un n-gramma?

Un n-gramma no es más que una subsecuencia de *n* elementos de una secuencia dada.

En nuestro caso estas subsecuencias serán de tres elementos.

#### ¿Cuál es es el objetivo de construir este tipo de modelo?

El objetivo con la construcción de este modelo es poder determinar cuál es la probabilidad de ocurrencia de una frase para usar esta probabilidad de ocurrencia en el momento de ranquear las sugerencias del corrector ortográfico. Un modelo n-gramma intenta predecir la próxima palabra de una oración a partir de las N-1 anteriores.

#### Estimación de probabilidades

Objetivo: calcular la probabilidad de que una palabra $w$ ocurra luego de una secuencia previa $h$. $P(w|h)$

La probabilidad se estima a partir de las frecuencias de ocurrencias en un corpus (Principio de máxima verosimilitud: ajustar lo mejor posible a los datos)

Entonces para una frase como:

    "Los cubanos somos alegres"
    
Podemos calcular $P$("alegres"|"Los cubanos somos"), si tomamos a $C$ como una función que cuenta la cantidad de ocurrencias de una frase tendríamos:

$P$("alegres"|"Los cubanos somos") = $C$("Los cubanos somos alegres)/$C$("Los cubanos somos")

O sea la división entre la cantidad de veces que aparece la expresión "Los cubanos somos alegres" y "Los cubanos somos".

Sin embargo hemos mencionado que utilizaremos un trigrama para modelar nuestro lenguaje por lo tanto solo podemos estimar la probabilidad de manera directa de una secuencia de 3 palabras:

"Los cubanos somos": $P$("somos"|"Los cubanos")

Pues para estimar la probabilidad de secuencias mas largas nos podemos valer de la **regla de la cadena**.

    "Los cubanos somos alegres"
    
$P$("somos"|"Los cubanos") * $P$("alegres"|"cubanos somos")

Y asi podemos extraer de cada frase sus trigrama correspondientes y calcular la probabilidad de ocurrencia de toda la frase basandonos en las probabilidades de ocurrencia de sus trigrama. En nuestro caso además disponemos de un caracter especial que nos indica el inicio y fin de la frase por lo que obtendríamos la probabilidad de ocurrencia de esta frase de la siguiente manera:

    "_ _ Los cubanos somos alegres _ _"
    
$P$("Los"|"_ \_") * $P$("cubanos"|"_ Los") * $P$("somos"|"Los cubanos") * $P$("alegres"|"cubanos somos") * $P$("\_"|"somos alegres") * $P$("_"|"alegres _")

##### Hipótesis Markoviana

Sin embargo en la práctica este método para calcular la probabilidad de ocurrencia de una frase (oración, expresión, sentencia) puede hacer que el cálculo sea muy costosos además de tener que lidiar con algunos problemas como la creatividad del lenguaje. Lo que haremos en este caso será en lugar de calcular la probabilidad de ocurrencia de una palabra $w$ dado un historial $h$ de palabras, podemos aproximar $h$ tomando en cuenta *las últimas palabras.* 

Para nuestro caso en particular empleando un trigrama tenemos que: $P(w_n|w_1^{n-1})$ se puede estimar como $P(w_n|w_{n-2}^{n-1})$

O sea: $P$("alegres"|"Los cubanos somos") se estimaria simplemente como $P$("alegres"|"cubanos somos")

#### Problema de las probabilidades nulas

Como hemos visto anteriormente la probabilidad de ocurrencia de una frase se calcula en base a cuantas veces ha sido visto un n-gramma en el corpus. Puede suceder que estemos procesando una frase que contenga n-grammas que no se hallan encontrado en el corpus, esto provocaría que la probabilidad de ese n-gramma se haga cero y por ende la probabilidad de toda la frase sea cero. Para evitar este problema utilizaremos una *técnica de suavizado*, para este caso la técnica consiste en asumir que todas las posibles combinaciones de las palabras de nuestro vocabulario han sido vistas al menos una vez, o sea todos los posibles trigramas que podamos formar tendrán un contador que será inicializado con 1. A partir de ahí se comienza a realizar el conteo en base a los n-grammas extraídos del corpus.

Repasaremos rapidamente el código del artículo anterior para tener nuestros datos cargados:

In [1]:
import re
import numpy as np

text = ''

for i in range(1, 202):
    text += (open('CORPUS/{}.txt'.format(i)).read().split('Contenido:')[-1])
    
def clean_text(text):
    text = re.sub(r'\n', '. ', text)
    text = re.sub(r'[{}@_*<>()\\#%+=\[\]ôà|è$–/—‘’«°º»”“&…∑⁄]', '', text)
    text = re.sub('a0', '', text)
    text = re.sub('\'92t', '\'t', text)
    text = re.sub('\'92s', '\'s', text)
    text = re.sub('\'92m', '\'m', text)
    text = re.sub('\'92ll', '\'ll', text)
    text = re.sub('\'91', '', text)
    text = re.sub('\'92', '', text)
    text = re.sub('\'93', '', text)
    text = re.sub('\'94', '', text)
    text = re.sub('\uf0b7', '', text)
    text = re.sub('\uf0e0', '', text)
    text = re.sub('\u200e', '', text)
    text = re.sub('\ufeff', '', text)
    text = re.sub('\u200b', '', text)
    text = re.sub('\t', '', text)
    text = re.sub('\r', '', text)
    text = re.sub('\.', '. ', text)
    text = re.sub('\!', '! ', text)
    text = re.sub('\?', '? ', text)
    text = re.sub(' +', ' ', text)
    
    return text

text = clean_text(text)

text = text.replace('.', ' _ _ ')

vocab = sorted(set(re.findall('[A-Za-z_áéíóúüñ]+', text)))

word2id = { word:i for i, word in enumerate(vocab) }

id2word = np.array(vocab)

#### Detalles de implementación

Para implementar nuestro modelo del lenguaje trigrama nos valdremos de una matriz cuadrada de tres dimensines con un tamaño igual a la longitud del vocabulario (cantidad de palabras) donde cada indice corresponde a una palabra del vocabulario, recordemos que ya tenemos un diccionario que nos permite obtener el id de una palabra, y según el orden de los índices será el orden en que aparece dicha palabra en nuestro trigrama. La matriz será inicializada con todos sus valores a 1 (técnica de suavizado) y se procederá a extraer del corpus los trigrama para ir contando la ocurrencia de cada uno de ellos.

In [2]:
model = np.ones((len(vocab), len(vocab), len(vocab)), dtype='uint')

MemoryError: Unable to allocate array with shape (12115, 12115, 12115) and data type uint64

#### EPIC FAIL

    "En teoría, la teoría y la práctica son lo mismo, pero en la práctica son muy diferentes"
    
Sucede que estamos tratando de crear una matriz de tres dimensiones con un tamaño inmenso. Lo que tenemos es una lista de elementos (cantidad de palabras tomadas del corpus) donde cada elemento es una matriz de `len(vocab)` x `len(voocab)`, y cada elemento de la matriz ocupa 64 bits. Lo que nos exige una capacidad de memoria de:

In [ ]:
print("{} GB".format((len(vocab)*len(vocab)*len(vocab)*64)/8/1024/1024/1024))

Si, son unos cuantos GB de memoria que por supuesto no tenemos a nuestra disposición. Entonces nos va tocar usar un método más eficiente para implementar nuestro modelo del lenguaje.

El error en este intento fallido ha sido tratar de modelar y asignar un valor a todas las posibles combinaciones de tres palabras que podemos formar con nuestro vocabulario. En lugar de eso utilizaremos otro tipo de estructura de datos para representar solamente los trigramas que sean encontrados en nuestro corpus, por lo que tendríamos una cantidad mucho menor de datos que almacenar en memoria. Esta vez utilizaremos diccionarios para almacenar la información deseada, podemos mencionar que los diccionarios en `python` se basan en grafos para el almacenamiento e indexación de los datos y dicha estructura debe ser mucho más eficiente para la tarea que nos proponemos realizar.

En el próximo artículo implementaremos nuestro modelo del lenguaje utilizando diccionarios. Extraeremos los trigramas del corpus y calcularemos sus probabilidades.